# <center><ins>Analise dos dados da tabela</ins></center>

## Fazendo os imports necessários

<p>A idéia é treinar um modelo para identificação do possível numero de horas que serão trabalhados para cada projeto.</p>
<p>Para isso, os dados que vem da tabela são agrupados e normalizados para um dataframe que vai virar um csv.</p></br>

> <p>Os dados são extraidos do mongo, transformados em dataframes e então carregados como arquivo csv.</p>

In [2]:
import pandas as pd
from database.db_connection import DatabaseConnection
from database.Repositories.AppointedHours_repository import AppointedHours_repository
from database.Repositories.ProjectPlanning_repository import ProjectPlanning_repository

<p> Transformando os registros do banco em dataFrames <p>

In [3]:
database = DatabaseConnection()
database.connect()

appointed_repository = AppointedHours_repository(database)
planning_repository = ProjectPlanning_repository(database)

appointments_df = pd.DataFrame(appointed_repository.get_all_appointed_hours())
plannings_df = pd.DataFrame(planning_repository.get_all_pplaning())

database.close()

app_df = appointments_df.copy()
ppl_df = plannings_df.copy()

### <center>Transformando as informações do dataframe.</center>

<p><ins>Schema de appointments_df</ins></p>

| **Campo**       | **Tipo de Dado**      |
|-----------------|-----------------------|
| _id             | object                |
| user            | object                |
| project         | object                |
| entry_id        | object                |
| start_at        | datetime64[ns]        |
| end_at          | datetime64[ns]        |
| worked_hours    | float64               |


In [4]:
columns = ['project', 'user', 'start_at','end_at','worked_hours']
app_df = app_df[columns]
app_df['period'] = app_df['start_at'].dt.to_period('M').astype(str)
app_df =  app_df.drop(['start_at', 'end_at'], axis=1)

### <center>Transformando as informações do dataframe.</center>

<p><ins>Schema de plannings_df</ins></p>


| **Campo**       | **Tipo de Dado**      |
|-----------------|-----------------------|
| _id             | object                |
| planning_id     | object                |
| project_id      | object                |
| date_start      | datetime64[ns]        |
| date_end        | datetime64[ns]        |
| contracted_hours| float64               |
| used_hours      | float64               |
| remaining_hours | float64               |

In [5]:
columns = ['project_id', 'date_start', 'date_end', 'contracted_hours']
ppl_df = ppl_df[columns]
ppl_df['period'] = ppl_df['date_start'].dt.to_period('M').astype(str)
ppl_df = ppl_df.drop(['date_start','date_end'], axis=1)

## <center>Modelar uma base de aprendizado<center>

In [6]:
columns = ['user', 'project', 'period', 'worked_hours', 'contracted_hours']
app_df = app_df.groupby(['user','project','period']).agg({
    'worked_hours': 'sum'
}).reset_index()

merged_df = pd.merge(app_df, ppl_df, left_on=['project', 'period'], right_on=['project_id', 'period'], how='inner')

merged_df = merged_df[columns].sort_values(by='period')

merged_df['remaining_hours'] = merged_df['contracted_hours'] - merged_df['worked_hours']

merged_df.to_csv(f"./datasets/training_datas{merged_df['period'].min()}_to_{merged_df['period'].max()}", index=False)